In [1]:
# change to tf 2.0 (run on colab)
# !pip uninstall tensorflow
# !pip install tensorflow==2.0.0

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
# Load git code
!git clone -s https://VincentHenric:@github.com/VincentHenric/Deep_learning_project.git
%cd Deep_learning_project
!ls

from google.colab import files

In [ ]:
# To use dynamic batches
!pip install pybind11
!./compile.sh

In [ ]:
# Access Drive
from google.colab import drive
drive.mount('/content/gdrive')

# Move data to working folder
%cp -a "../gdrive/My Drive/deep_learning_project/" "."
%ls

In [3]:
%load_ext autoreload
%autoreload 1

import numpy as np
from matplotlib import pyplot as plt

import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
import golois

%aimport models.py
print(tf.__version__)

2.0.0


In [ ]:
PLANES = 8
MOVES = 361
DYNAMIC_BATCH = False
model_name = ""

In [ ]:
if DYNAMIC_BATCH:
    N = 100000
    input_data = np.random.randint(2, size=(N, 19, 19, PLANES))
    input_data = input_data.astype ('float32')
    
    policy = np.random.randint(MOVES, size=(N,))
    policy = keras.utils.to_categorical (policy)
    
    value = np.random.randint(2, size=(N,))
    value = value.astype ('float32')
    
    end = np.random.randint(2, size=(N, 19, 19, 2))
    end = end.astype ('float32')

    golois.getBatch (input_data, policy, value, end)
else:
    input_data = np.load ('input_data.npy')
    policy = np.load ('policy.npy')
    value = np.load ('value.npy')
    end = np.load ('end.npy')

In [ ]:
models = os.listdir('models')
models = [file[:-3] for file in models if file.endswith('.h5')]
if model_name in models:
    model = model.load('{}.h5'.format(model_name))
else:
    model = models.get_model_week2(128)

model.summary()

model.compile(optimizer=keras.optimizers.SGD(lr=0.1),
              loss={'value': 'mse', 'policy': 'categorical_crossentropy'})

In [ ]:
checkpointer = ModelCheckpoint(filepath='models/{}.h5'.format(model_name),
                                   verbose=1,
                                   save_best_only=True)

print('Start learning')
history = model.fit(input_data, {'policy': policy, 'value': value},
          epochs=20, batch_size=128, validation_split=0.1,
          callbacks=[checkpointer])

In [ ]:
# Download the model
files.download('models/{}.h5'.format(model_name))

In [6]:
model = keras.models.load_model('models/HENRIC.h5')